Checking if CUDA is enabled first.

In [1]:
# Check if cuda is enabled 
import torch
import os
os.environ["KERAS_BACKEND"]="tensorflow" # options 'torch' / 'jax' / 'tensorflow'
torch.cuda.is_available()

True

Dataset for fine-tuning

In [2]:
# Loading dataset
import json
data=[]
with open('./model/dataset/databricks-dolly-15k.jsonl') as file:
    for line in file:
        features=json.loads(line)
        template="Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))

# Need permission from Gemma first
# Loading Huggingface Token

import json

with open("token.json", "r") as json_file:
    token_dict = json.load(json_file)

access_token = token_dict["token"]
from huggingface_hub import login
login(token=access_token)

/home/devy/miniconda3/envs/AI/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/devy/.var/app/com.visualstudio.code/cache/huggingface/token
Login successful


Using Gemma 2B from HuggingFace

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
cache_dir='./model/gemma'
PYTORCH_CUDA_ALLOC_CONF=expandable_segments=True
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b",device_map="auto",cache_dir=cache_dir)
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b",cache_dir=cache_dir)
model.to('cuda')

Loading checkpoint shards: 100%|██████████| 2/2 [00:58<00:00, 29.10s/it]


Fine-Tuning<br>
Option 1: Low rank adaptation

In [ ]:
# Using LORA

from peft import LoraConfig

# LoRA config
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

import transformers
from trl import SFTTrainer

def formatting_func(example):
    output_texts = []
    for i in range(len(example)):
        text = f"Instruction: {example['instruction'][i]}\nResponse: {example['response'][i]}"
        output_texts.append(text)
    return output_texts


trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)
trainer.train()


2024-03-18 14:24:12.416753: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-18 14:24:13.507538: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-2b.
401 Client Error. (Request ID: Root=1-65f8326f-7f41867b4ded87d20513c5a6;ee4288ff-f0aa-433f-b97e-49be24b3b6a7)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b/resolve/main/config.json.
Repo model google/gemma-2b is gated. You must be authenticated to access it.

Response

In [ ]:
text = "Hello, how are you?"
input = tokenizer(text, return_tensors="pt").to("cuda")

outputs = model.generate(**input)
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

Alternative Response 

In [ ]:
%%script echo skipping
prompt=template.format(
    instruction="Can you introduce yourself?",
    response="",
)
print(model.generate(prompt,max_length=256))